# Data to df

In [39]:
import requests
import bs4
import pandas as pd
import re
import numpy as np

res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = bs4.BeautifulSoup(res.text, 'html.parser')

h3 = soup.select('tr')
rows=[]

for i in h3:
    rows.append(i.text)

k1 = []

for i in range(len(h3)):
    k = h3[i].text
    k1.append(re.findall(r"[\w ']+", k))

df = pd.DataFrame(data = k1)

df.columns = df.iloc[0]

df.drop(df.index[0],inplace=True)

df.drop(df.index[180:],inplace=True)


# Deleting Not assigned from Borough

In [40]:
indexnames = df[df['Borough']=='Not assigned'].index

df.drop(indexnames,inplace=True)

# Combining Neighbourhoods

In [41]:
df.astype('str')

df = df.replace({None:''})

df['Neighborhood'] = df.iloc[:,2:].apply(lambda x: ','.join(x), axis = 1)

df.head()

,Postal Code,Borough,Neighborhood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,M3A,North York,"Parkwoods,,,,,,,,,,,,,,,,,,,,,,,,,,,,",,,,,,,,...,,,,,,,,,,
4,M4A,North York,"Victoria Village,,,,,,,,,,,,,,,,,,,,,,,,,,,,",,,,,,,,...,,,,,,,,,,
5,M5A,Downtown Toronto,"Regent Park, Harbourfront,,,,,,,,,,,,,,,,,,,,,...",Harbourfront,,,,,,,...,,,,,,,,,,
6,M6A,North York,"Lawrence Manor, Lawrence Heights,,,,,,,,,,,,,,...",Lawrence Heights,,,,,,,...,,,,,,,,,,
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government,,,...",Ontario Provincial Government,,,,,,,...,,,,,,,,,,


# Deleting Remaining Columns

In [42]:
df.drop(columns=df.iloc[:,3:],axis=1,inplace=True)

df.head()

,Postal Code,Borough,Neighborhood
3,M3A,North York,"Parkwoods,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
4,M4A,North York,"Victoria Village,,,,,,,,,,,,,,,,,,,,,,,,,,,,"
5,M5A,Downtown Toronto,"Regent Park, Harbourfront,,,,,,,,,,,,,,,,,,,,,..."
6,M6A,North York,"Lawrence Manor, Lawrence Heights,,,,,,,,,,,,,,..."
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government,,,..."


# Deleting extra terms in neighborhood

In [43]:
df['Neighborhood'] = df['Neighborhood'].str.split(',,').str[0]

df.head()

,Postal Code,Borough,Neighborhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M6A,North York,"Lawrence Manor, Lawrence Heights"
7,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [44]:
df.reset_index(drop=True)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Shape

In [45]:
df.shape

(103, 3)

# Adding Coordinates using Geocoder

In [67]:
cord = pd.read_csv("Geospatial_Coordinates.csv")


In [51]:
cord.shape

(103, 3)

In [72]:
df2 = pd.merge(df,cord,on='Postal Code')

df2.shape

In [79]:
df2.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
